## Vincular a drive

In [ ]:
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
os.chdir("drive/MyDrive/Aprendizaje/UOC/2023_2024_S5/Git/TFM")

## Importar librerías

In [ ]:
from src.gan import GAN
from src.generator import generator
from src.discriminator import  discriminator

In [ ]:
dcgan = GAN(generator =  generator(additional_blocks=3).generator_model,
            discriminator = discriminator(additional_layers=4, depth = 128).discriminator_model,
            path_resultados = "Prueba",
            # data_dir = "Datos/data_npz_64x64_compressed/",
            epochs = 3,
            sample_interval= 1,
            batch_size=128,
            learning_rate=0.0005)

dcgan

Se han cargado 21554 imagenes.
Carpeta 'Prueba' creada exitosamente.


In [ ]:
dcgan.train()

0.0042133331298828125
4/4 [==============================] - 0s 17ms/step - loss: 0.6883 - accuracy: 1.0000
0 [D loss: 0.740227, reals_acc.: 100.00%, false_acc.:100.00%] [G loss: 0.254433 - 1.00%]
32.505643367767334
1/1 [==============================] - 1s 832ms/step
1.3273723125457764
4/4 [==============================] - 0s 17ms/step - loss: 0.6409 - accuracy: 1.0000
1 [D loss: 0.419808, reals_acc.: 99.22%, false_acc.:100.00%] [G loss: 0.063155 - 1.00%]
1.4464054107666016
1/1 [==============================] - 0s 17ms/step
0.290097713470459
4/4 [==============================] - 0s 16ms/step - loss: 0.3249 - accuracy: 1.0000
2 [D loss: 0.320751, reals_acc.: 100.00%, false_acc.:100.00%] [G loss: 1.308977 - 0.04%]
1.3611664772033691
1/1 [==============================] - 0s 17ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
